This notebook will create an application from the SVM model and makes it ready to deploy online.

In [9]:
# imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.dummy import DummyRegressor
from sklearn.svm import SVR

import pickle
import gradio as gr

### Load pipeline

In [19]:
pipeline = pickle.load(open("model.pkl", 'rb'))

In [20]:
# Load data
math_df = pd.read_csv('../Data/Math_Test_Results_Cleaned.csv')
X = math_df.drop('Mean Scale Score', axis=1)
y = math_df['Mean Scale Score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [22]:
pipeline.score(X_test, y_test)

0.979721351891418

The pipeline has been loaded successfully!

### Deploy with Gradio
First of all, gradio needs a callable function.  
This is the prediction function in my case. 

In [96]:
def predict_score(grade, year, n_lvl1, n_lvl2, n_lvl3, n_lvl4, DBN):
    n_tested = n_lvl1 + n_lvl2 + n_lvl3 + n_lvl4
    
    pct_lvl1 = 0 if n_lvl1<=0 else (n_lvl1 / n_tested) * 100
    pct_lvl2 = 0 if n_lvl2<=0 else (n_lvl2 / n_tested) * 100
    pct_lvl3 = 0 if n_lvl3<=0 else (n_lvl3 / n_tested) * 100
    pct_lvl4 = 0 if n_lvl4<=0 else (n_lvl4 / n_tested) * 100
    
    district_n = DBN[:2]
    borough_n = DBN[2]
    encoding = {'M':0, 'X':1, 'K':2, 'Q':3, 'R':4}
    borough_n = encoding[borough_n]
    school_n = DBN[3:]
    
    X = {"Grade":grade, "Year":year, "Number Tested":n_tested, "Num Level 1":n_lvl1, "Pct Level 1":pct_lvl1,
         "Num Level 2":n_lvl2, "Pct Level 2":pct_lvl2, "Num Level 3":n_lvl3, "Pct Level 3":pct_lvl3,
         "Num Level 4":n_lvl4, "Pct Level 4":pct_lvl4, "District Number":district_n, 
         "Borough Number":borough_n, "School Number":school_n}

    y = pipeline.predict(pd.DataFrame(X, index=[0]))
    return int(y[0])

We also need some input fields to get information from the user.

In [94]:
#create input and output objects
grade = gr.Dropdown([3, 4, 5, 6, 7, 8], label="Grade")
year = gr.Dropdown([2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016], label="Year")
n_lvl1 = gr.Number(precision=0, label="Amount of students in Level 1")
n_lvl2 = gr.Number(precision=0, label="Amount of students in Level 2")
n_lvl3 = gr.Number(precision=0, label="Amount of students in Level 3")
n_lvl4 = gr.Number(precision=0, label="Amount of students in Level 4")
DBN = gr.Textbox(label="DBN (e.g. '01M188')")

#output object
output = gr.Textbox(label="Predicted Mean Scale Score")

{'Grade': 5, 'Year': 2007, 'Number Tested': 68, 'Num Level 1': 20, 'Pct Level 1': 29.411764705882355, 'Num Level 2': -3, 'Pct Level 2': -4.411764705882353, 'Num Level 3': 11, 'Pct Level 3': 16.176470588235293, 'Num Level 4': 40, 'Pct Level 4': 58.82352941176471, 'District Number': '01', 'Borough Number': 0, 'School Number': '188'}


In [63]:
X_test.iloc[0]

Grade                 7.0
Year               2012.0
Number Tested       412.0
Num Level 1           0.0
Pct Level 1           0.0
Num Level 2          22.0
Pct Level 2           5.3
Num Level 3          97.0
Pct Level 3          23.5
Num Level 4         293.0
Pct Level 4          71.1
District Number      21.0
Borough Number        2.0
School Number        98.0
Name: 15000, dtype: float64

In [95]:
gui = gr.Interface(fn=predict_score,
                   inputs=[grade, year, n_lvl1, n_lvl2, n_lvl3, n_lvl4, DBN],
                   outputs=[output])
gui.launch()

Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.


In [97]:
print(predict_score(grade=5, year=2015, n_lvl1=10, n_lvl2=20, n_lvl3=20, n_lvl4=50, DBN="01M188"))

689


Example rows from the test set for testing the online deployed model:  

[Grade, Year, Lvl1, Lvl2, Lvl3, Lvl4, DBN] -> result 

[6, 2011, 3, 27, 23, 8, 12X271] -> 674  
[3, 2006, 20, 24, 38, 16, 22K109] -> 656